In [46]:
%pip install kubernetes
%pip install humanfriendly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
from humanfriendly import format_size
from kubernetes import client, config
import json
config.load_kube_config()

v1 = client.CoreV1Api()

In [83]:
# https://github.com/kubernetes-client/python/blob/master/kubernetes/utils/quantity.py
from kubernetes.utils import quantity
quantity.parse_quantity("40Mi")
quantity.parse_quantity("500m")

Decimal('0.500')

In [101]:
def get_pod_usage(containers):
  memory = 0
  cpu = 0
  if isinstance(containers, list) and isinstance(containers[0], dict):
    assert len(containers) <= 2, "Only 2 containers allowed"
    for c in containers:
      cpu += quantity.parse_quantity(c['usage']['cpu'])
      memory += quantity.parse_quantity(c['usage']['memory'])
  else:
    for c in containers:
      if 'istio-proxy' in c.name:
        continue
      limits = c.resources.limits
      if limits is None:
        continue
      if 'cpu' in limits:
        cpu += quantity.parse_quantity(limits['cpu'])
      else:
        cpu += 0

      if 'memory' in limits:
        memory += quantity.parse_quantity(limits['memory'])
      else:
        memory += 0
  return dict(cpu = cpu, memory = memory)

In [103]:
class DeploymentResourceStatsEncoder(json.JSONEncoder):

  def default(self, o):
    return dict(
          app_name=o._app_name,
          cpu_usage=o.cpu_usage(),
          memory_usage=o.memory_usage(),
          cpu_limit = o.cpu_limit(),
          memory_limit = o.memory_limit()
        )

class DeploymentResourceStats(object):
  def __init__(self, app_name):
    super().__init__()
    self._app_name = app_name
    self._metrics = []
  
  def add_metric_from_containers(self, container_limits, container_usages):
    limits = get_pod_usage(container_limits)
    usage = get_pod_usage(container_usages)
    cpu_usage = usage['cpu'] * 1000
    memory_usage = usage['memory']
    cpu_limit = limits['cpu'] * 1000
    memory_limit = limits['memory']
    self._metrics.append(dict(
      cpu_limit = cpu_limit,
      memory_limit = memory_limit,
      cpu_usage = cpu_usage,
      memory_usage = memory_usage
    ))

  def cpu_usage(self):
    cpu = 0
    for metric in self._metrics:
      cpu += metric['cpu_usage']
    return cpu

  def memory_usage(self, value=False):
    memory = 0
    for metric in self._metrics:
      memory += metric['memory_usage']

    if value:
      return memory
    else:
      return format_size(memory, binary=True)

  def cpu_limit(self):
    cpu = 0
    for metric in self._metrics:
      cpu += metric['cpu_limit']

    if int(cpu) == 0:
      cpu = 0xffffffff
    return cpu

  def memory_limit(self, value=False):
    memory = 0
    for metric in self._metrics:
      memory += metric['memory_limit']

    if int(memory) == 0:
      memory = 0xffffffff
    if value:
      return memory
    else:
      return format_size(memory, binary=True)

  def __repr__(self):
    return f'app: {self._app_name}, cpu_rate: {round(self.cpu_usage()/self.cpu_limit(), 2)}, memory_rate: {round(self.memory_usage(True)/self.memory_limit(True),2)}, cpu_usage: {self.cpu_usage()}, memory_usage: {self.memory_usage()}, cpu_limit: {self.cpu_limit()}, memory_limit: {self.memory_limit()}'

  def __str__(self):
    return self.__repr__()


In [113]:
# kubectl get --raw "/apis/metrics.k8s.io/v1beta1/nodes"
# https://kubernetes.io/docs/tasks/debug/debug-cluster/resource-metrics-pipeline/
api = client.CustomObjectsApi()
resource = api.list_namespaced_custom_object(group="metrics.k8s.io",version="v1beta1", namespace="default", plural="pods")


In [129]:
stat_list = {}
for pod in resource['items']:
  pod_name = pod['metadata']['name']
  if 'app' not in pod['metadata']['labels'] and 'run' not in pod['metadata']['labels']:
    continue
  app_name = pod['metadata']['labels'].get('app', pod['metadata']['labels'].get('run'))
  print(app_name)
  target_pod = v1.read_namespaced_pod(pod_name, namespace='default')
  container_limits = target_pod.spec.containers
  container_usages = pod['containers']
  stats = stat_list.get(app_name, DeploymentResourceStats(app_name))
  stats.add_metric_from_containers(container_limits, container_usages)
  stat_list[app_name] = stats

stat_list

details
fortio
httpbin
httpbin
httpbin
load-generator
php-apache
productpage
ratings
reviews
reviews
reviews


{'details': app: details, cpu_rate: 0.00, memory_rate: 0.01, cpu_usage: 1.783075000, memory_usage: 49.64 MiB, cpu_limit: 4294967295, memory_limit: 4 GiB,
 'fortio': app: fortio, cpu_rate: 0.00, memory_rate: 0.01, cpu_usage: 2.133090000, memory_usage: 38.37 MiB, cpu_limit: 4294967295, memory_limit: 4 GiB,
 'httpbin': app: httpbin, cpu_rate: 0.00, memory_rate: 0.05, cpu_usage: 4.572519000, memory_usage: 209.99 MiB, cpu_limit: 4294967295, memory_limit: 4 GiB,
 'load-generator': app: load-generator, cpu_rate: 0.00, memory_rate: 0.01, cpu_usage: 11.220326000, memory_usage: 34.28 MiB, cpu_limit: 4294967295, memory_limit: 4 GiB,
 'php-apache': app: php-apache, cpu_rate: 0.84, memory_rate: 0.01, cpu_usage: 417.974698000, memory_usage: 54.7 MiB, cpu_limit: 500.000, memory_limit: 4 GiB,
 'productpage': app: productpage, cpu_rate: 0.00, memory_rate: 0.02, cpu_usage: 6.681963000, memory_usage: 64.37 MiB, cpu_limit: 4294967295, memory_limit: 4 GiB,
 'ratings': app: ratings, cpu_rate: 0.00, memory_r

In [ ]:
# Scale up
apiAppsV1 = client.AppsV1Api() # Patch scale


In [130]:
php_apache = apiAppsV1.read_namespaced_deployment_scale	('php-apache', "default")
print(f'current replicas: {php_apache.spec.replicas}')


current replicas: 3


In [116]:
php_apache.spec.replicas += 1
apiAppsV1.patch_namespaced_deployment_scale('php-apache', "default", body=php_apache.to_dict())

{'api_version': 'autoscaling/v1',
 'kind': 'Scale',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2022, 5, 16, 12, 38, 57, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'php-apache',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '1209124',
              'self_link': None,
              'uid': 'acac33bd-aba5-4b07-ac13-ba2028163f5b'},
 'spec': {'replicas': 3},
 'status': {'replicas': 2, 'selector': 'run=php-apache'}}

In [81]:
# resource return by `list_namespaced_custom_object` is a dict
for pod in resource["items"]:
  pod_name = pod['metadata']['name']
  if 'app' in pod['metadata']['labels']:
    label_app = pod['metadata']['labels']['app']
  elif 'run' in pod['metadata']['labels']:
    label_app = pod['metadata']['labels']['run']
  else:
    label_app = pod_name
  # target_pod is a Pod object
  target_pod = v1.read_namespaced_pod(pod_name, namespace='default')
  limits = get_pod_usage(target_pod.spec.containers)
  usage = get_pod_usage(pod['containers'])
  cpu_usage = usage['cpu'] * 1000
  memory_usage = usage['memory'] / 1024 / 1024
  limit_cpu = limits['cpu'] * 1000
  limit_memory = limits['memory']  / 1024 / 1024
  print(f"app: {label_app}, name: {pod_name}, cpu: {cpu_usage}/{limit_cpu}, memory: {memory_usage}/{limit_memory} " )
  print(f"app: {label_app}, name: {pod_name}, cpu: {round(cpu_usage)}, memory: {round(memory_usage)} " )
  print(f"app: {label_app}, percent: {cpu_usage / limit_cpu}, {memory_usage /limit_memory}")

app: counter, name: counter, cpu: 1.565690000/2000, memory: 35.69921875/1024 
app: counter, name: counter, cpu: 2, memory: 36 
app: counter, percent: 0.000782845, 0.034862518310546875
app: details, name: details-v1-5498c86cf5-4wxd7, cpu: 1.856215000/2000, memory: 49.53125/1024 
app: details, name: details-v1-5498c86cf5-4wxd7, cpu: 2, memory: 50 
app: details, percent: 0.0009281075, 0.048370361328125
app: fortio, name: fortio-deploy-7dcd84c469-f2rdm, cpu: 1.635381000/2000, memory: 38.42578125/1024 
app: fortio, name: fortio-deploy-7dcd84c469-f2rdm, cpu: 2, memory: 38 
app: fortio, percent: 0.0008176905, 0.037525177001953125
app: httpbin, name: httpbin-85d76b4bb6-5fptx, cpu: 1.284320000/2000, memory: 72.55078125/1024 
app: httpbin, name: httpbin-85d76b4bb6-5fptx, cpu: 1, memory: 73 
app: httpbin, percent: 0.000642160, 0.070850372314453125
app: httpbin, name: httpbin-85d76b4bb6-6ggpq, cpu: 1.239647000/2000, memory: 71.703125/1024 
app: httpbin, name: httpbin-85d76b4bb6-6ggpq, cpu: 1, memo

In [98]:
target_pod = v1.read_namespaced_pod('php-apache-7656945b6b-75fzf', namespace='default')
get_pod_usage(target_pod.spec.containers)

{'cpu': Decimal('2.500'), 'memory': Decimal('1073741824')}

In [93]:
limits

{'cpu': Decimal('2'), 'memory': Decimal('1073741824')}

In [92]:
target_pod.spec

{'active_deadline_seconds': None,
 'affinity': None,
 'automount_service_account_token': None,
 'containers': [{'args': None,
                 'command': None,
                 'env': None,
                 'env_from': None,
                 'image': 'k8s.gcr.io/hpa-example',
                 'image_pull_policy': 'Always',
                 'lifecycle': None,
                 'liveness_probe': None,
                 'name': 'php-apache',
                 'ports': [{'container_port': 80,
                            'host_ip': None,
                            'host_port': None,
                            'name': None,
                            'protocol': 'TCP'}],
                 'readiness_probe': None,
                 'resources': {'limits': {'cpu': '500m'},
                               'requests': {'cpu': '200m'}},
                 'security_context': None,
                 'startup_probe': None,
                 'stdin': None,
                 'stdin_once': None,
             

In [93]:
resource["items"][0]

{'metadata': {'name': 'counter',
  'namespace': 'default',
  'creationTimestamp': '2022-05-12T03:23:18Z',
  'labels': {'security.istio.io/tlsMode': 'istio',
   'service.istio.io/canonical-name': 'counter',
   'service.istio.io/canonical-revision': 'latest'}},
 'timestamp': '2022-05-12T03:23:12Z',
 'window': '15.927s',
 'containers': [{'name': 'istio-proxy',
   'usage': {'cpu': '2171530n', 'memory': '35536Ki'}},
  {'name': 'count', 'usage': {'cpu': '918113n', 'memory': '1664Ki'}}]}

In [122]:
# Resources set by user
# Pod is Pod object
pod_name = resource["items"][0]['metadata']['name']
pod = v1.read_namespaced_pod(pod_name, namespace='default')
get_pod_usage(pod.spec.containers)
# pod.spec.containers[0].resources

{'cpu': Decimal('2'), 'memory': Decimal('1073741824')}

In [95]:
usage = get_pod_usage(resource["items"][0])
cpu_usage = usage['cpu'] * 1000
memory_usage = usage['memory'] / 1024 / 1024
print(f"name: {pod_name}, cpu: {cpu_usage}, memory: {memory_usage} " )
print(f"name: {pod_name}, cpu: {round(cpu_usage)}, memory: {round(memory_usage)} " )

name: counter, cpu: 3.089643000, memory: 36.328125 
name: counter, cpu: 3, memory: 36 


In [115]:
ret = v1.list_namespaced_pod(namespace='default')
ret.items[0].spec.containers[-1].resources.limits

{'cpu': '2', 'memory': '1Gi'}